In [34]:
from abc import ABC, abstractmethod
import uuid
import json
from itertools import groupby
from operator import itemgetter

lin

In [35]:
class TransformationStrategy(ABC):
    
    @abstractmethod
    def transform(self, timesheet):
        pass
    
    @staticmethod
    def generate_unique_id():
        return uuid.uuid4()


In [36]:
class GroupByWellStrategy(TransformationStrategy):
    def transform(self, timesheet):
        grouped_line_items = {}
        
        for key, group in groupby(timesheet['lineItems'], key=itemgetter('well')):
            grouped_line_items[key] = sum(item['amount'] for item in group)

        aggregated_timesheet = {
            'id': str(self.__class__.generate_unique_id()),
            'timesheet': timesheet['timesheetId'],
            'line_items': grouped_line_items
        }
                       
        return aggregated_timesheet


In [37]:
# groupByWellStrategy = GroupByWellStrategy()
# output_data = groupByWellStrategy.transform(data[0])
# print(json.dumps(output_data, indent=2))

In [38]:
class MergeToOneLineItemStrategy(TransformationStrategy):
    def transform(self, timesheet):
        transformed_timesheets = []
        
        total_amount = 0
        
        for line_item in timesheet['lineItems']:
            total_amount += line_item['amount']
            
        return {
            'id': str(GroupByWellStrategy.generate_unique_id()),
            'timesheetId': timesheet['timesheetId'],
            'totalAmount': total_amount
        }


In [39]:
# mergeToOneLineItemStrategy = MergeToOneLineItemStrategy()
# output_data = mergeToOneLineItemStrategy.transform(data[0])
# print(json.dumps(output_data, indent=2))

In [40]:
class InvoicePerLineItemStrategy(TransformationStrategy):
    def transform(self, timesheet):
        transformed_timesheets = []
        
        total_amount = 0
        
        for line_item in timesheet['lineItems']: 
            transformed_timesheets.append({
                'id': str(GroupByWellStrategy.generate_unique_id()),
                'timesheetId': timesheet['timesheetId'],
                'well': line_item['well'],
                'totalAmount': line_item['amount']
            })
            
        return transformed_timesheets
    

In [41]:
# mergeToOneLineItemStrategy = InvoicePerLineItemStrategy()
# output_data = mergeToOneLineItemStrategy.transform(data[0])
# print(json.dumps(output_data, indent=2))

In [42]:
from enum import Enum

class LineItemStrategy(Enum):
    GROUPBY_WELL_STRATEGY = 1
    MERGE_TO_ONE_LINE_ITEM = 2
    INVOICE_PER_LINE_ITEM = 3

enum_dict = {
    LineItemStrategy.GROUPBY_WELL_STRATEGY: GroupByWellStrategy(), 
    LineItemStrategy.MERGE_TO_ONE_LINE_ITEM: MergeToOneLineItemStrategy(), 
    LineItemStrategy.INVOICE_PER_LINE_ITEM: InvoicePerLineItemStrategy()
#   other more complicated strategy such as 1 invoice per month (includes line items for all workers)
}

# assume this is retrieved from database
client_dict = {
    "client-01": LineItemStrategy.GROUPBY_WELL_STRATEGY,
    "client-02": LineItemStrategy.MERGE_TO_ONE_LINE_ITEM,
    "client-03": LineItemStrategy.INVOICE_PER_LINE_ITEM
    
}


In [43]:
with open('timesheets.json', 'r') as file:
    data = json.load(file)

for timesheet in data:
    client_id = timesheet['clientId']
    stratgy_enum = client_dict[client_id]
    strategy = enum_dict[stratgy_enum]
    result = strategy.transform(timesheet)
    
    print('_'*50)
    print()
    print(json.dumps(result, indent=1))
    print()
print('_'*50)

__________________________________________________

{
 "id": "e11744e9-005e-4d84-bdeb-d72a6c6b2a21",
 "timesheet": "TS001",
 "line_items": {
  "Well-01": 4651.25,
  "Well-02": 3525.75,
  "Well-03": 2775.75,
  "Well-04": 1550.25
 }
}

__________________________________________________

{
 "id": "7f608efe-38ae-42d8-9c06-f6c2a8cb770c",
 "timesheetId": "TS002",
 "totalAmount": 6526.5
}

__________________________________________________

[
 {
  "id": "da3dca19-affb-4e95-9176-1102ccc78df8",
  "timesheetId": "TS003",
  "well": "Well-01",
  "totalAmount": 1000.0
 },
 {
  "id": "29ea44c8-ab10-46a3-824a-e47ee2795299",
  "timesheetId": "TS003",
  "well": "Well-01",
  "totalAmount": 1200.5
 },
 {
  "id": "a7bf973f-f16e-4c14-8614-b4cfcec0745c",
  "timesheetId": "TS003",
  "well": "Well-03",
  "totalAmount": 1325.75
 },
 {
  "id": "fe4a4ac0-1677-487e-86c7-d577e70da59a",
  "timesheetId": "TS003",
  "well": "Well-03",
  "totalAmount": 1450.0
 },
 {
  "id": "fa25fc1b-58d7-4671-a71e-d574d9379108",
  "t